In [1]:
! pip install -q hazm
! pip install -q emoji

In [24]:
import pandas as pd
from pathlib import Path
import re
from collections import Counter
import joblib

import hazm
import emoji

In [4]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [5]:
normalizer = hazm.Normalizer()

def normalize(text):
    return normalizer.normalize(str(text))

def remove_emojis(text):
    return re.sub(emoji.get_emoji_regexp(), " ", str(text))

def remove_extra_spaces(text):
    return ' '.join([token for token in str(text).split()])

def remove_punctuation(text, exclude_list=None, trim_spaces=True):
    preserve_list = "^0-9\u0621-\u063F\u0641-\u064A\u0661-\u0669\u06F0-\u06F9\u06A9\u06AF\u06BE\u06C1\u06CC" \
                    "\u06D5\uFB8A\u067E\u0686\u0698"
    if type(exclude_list) == list and len(exclude_list) > 0:
        exclude_list_str = ''.join([str(token) for token in exclude_list])
    else:
        exclude_list_str = ''
    rule_str = f"[{preserve_list + exclude_list_str}]"
    rule = re.compile(rule_str)
    punc_removed = rule.sub(' ', str(text))
    if trim_spaces:
        return remove_extra_spaces(punc_removed)
    return punc_removed

def process(text):
    text = normalize(text)
    text = remove_emojis(text)
    text = remove_punctuation(text)
    return text

In [6]:
data_dir = Path('/content/drive/MyDrive/ContentAI/Data')
wiki_txt = data_dir / 'WIKI/WikiText.txt'
wiki_csv = data_dir / 'WIKI/WikiText.csv'

In [7]:
df = pd.read_csv(wiki_csv)
df.shape

(71600, 1)

In [8]:
df.head()

,text
0,تالار اجتماعات مربوط به دوره معاصر است و در شه...
1,خانه یک اتاقه مربوط به دوره معاصر است و در شهر...
2,انبار برنج مربوط به دوره معاصر است و در شهرستا...
3,بنای چایخانه موزه میراث روستایی مربوط به دوره ...
4,تپه گرجیان مربوط به هزاره ۴ قبل از میلاد تا هز...


In [11]:
df.loc[13213, 'text']

'سوبارو ویویو (Subaru Vivio) خودرویی است که در سال های ۱۹۹۲–۱۹۹۸تولید شده است. این خودرو در کلاس خودرو کی قرار گرفته، طراحی آن موتور جلو، خودرو محور جلو، خودرو چهار چرخ محرک بوده طول آن در حالت طبیعی ، عرض آن در حالت طبیعی ، ارتفاع آن در حالت طبیعی ، وزن آن در حالت طبیعی ، است.سیستم جعبه دندهٔ آن ۳، دنده به دو صورت خودکار و دستی است.'

In [12]:
process(df.loc[13213, 'text'])

'سوبارو ویویو خودرویی است که در سال های ۱۹۹۲ ۱۹۹۸تولید شده است این خودرو در کلاس خودرو کی قرار گرفته طراحی آن موتور جلو خودرو محور جلو خودرو چهار چرخ محرک بوده طول آن در حالت طبیعی عرض آن در حالت طبیعی ارتفاع آن در حالت طبیعی وزن آن در حالت طبیعی است سیستم جعبه دنده آن ۳ دنده به دو صورت خودکار و دستی است'

In [13]:
%%time

lines = df.text.to_list()
lines = [process(line) for line in lines]

CPU times: user 15min 52s, sys: 2.08 s, total: 15min 54s
Wall time: 15min 56s


In [15]:
words = [word for line in lines for word in re.findall(r'\w+', line)]
words = [word for word in words if not any(c.isdigit() for c in word) and len(word) > 1]
len(words)

31747056

In [16]:
words[:5]

['تالار', 'اجتماعات', 'مربوط', 'به', 'دوره']

In [17]:
words[34000:34010]

['دوران',
 'های',
 'تاریخی',
 'پس',
 'از',
 'اسلام',
 'است',
 'در',
 'شهرستان',
 'ایرانشهر']

In [25]:
freq = dict(Counter(words))
len(freq)

300547

In [33]:
joblib.dump(freq, 'vocab_freq.gz', compress=7)

['vocab_freq.gz']